In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/DL/Group/AdvPFY')

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torchvision.models.resnet import BasicBlock, resnet18, resnet50
import torch.optim as optim
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from pgd_purify import vae_purify, stae_purify, pgd_linf
# from model.nn_model import ResNetEnc, ResNetVAE
# revised
from model.nn_model_se import ResNetEnc, ResNetVAE

import random
import os

def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(0)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
batch_size = 256
# epoch_num = 2048
epoch_num = 100
lr_decay_step = 1024
classification_weight = 2048
vae_beta = 1

Using device: cuda:0


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor()])

random_transforms_list = transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter(),
                                    transforms.RandomAffine((-20, 20),
                                    translate=(0.0, 0.1), scale=(0.9, 1.1),
                                    fill=0.5)]), p=0.3)

aug_transform = transforms.Compose(
    [transforms.ToTensor(), transforms.RandomHorizontalFlip(p=0.5), random_transforms_list])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=aug_transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=1)

100%|██████████| 170M/170M [00:13<00:00, 12.5MB/s]


In [ ]:
net = ResNetEnc(image_size=32)
net = net.to(device)

ResVAE = ResNetVAE(net).to(device)
ResVAE = ResVAE.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 240MB/s]


In [ ]:
CE_Loss = nn.CrossEntropyLoss()
mseloss = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(ResVAE.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[lr_decay_step], gamma=0.1)

In [ ]:
test_acc = 0
pbar = tqdm(range(epoch_num))
for epoch in pbar:
    loss_sum = 0
    ResVAE.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # VAE
        x_reconst, z, y, mu, log_var = ResVAE(data, deterministic=False, classification_only=False)
        recons_loss = torch.sum((x_reconst - data) ** 2)
        kld_loss = -0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp())
        # jointly training
        loss_val = CE_Loss(y, target) * classification_weight + recons_loss + vae_beta * kld_loss
        loss_sum += loss_val.item()
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()


    scheduler.step()
    ResVAE.eval()

    # eval on test
    pred_list = []
    gt_list = []
    for batch_idx, (data, target) in enumerate(test_loader):
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            x_reconst, z, y_test, mu, log_var = ResVAE(data, deterministic=True, classification_only=False)

        pred_list += list(y_test.argmax(-1).cpu().detach().numpy())
        gt_list += list(target.detach().cpu().numpy())

    test_acc = np.sum(np.array(gt_list) == np.array(pred_list)) / len(gt_list)
    pbar.set_postfix({"train loss sum": loss_sum, "test acc": test_acc})

100%|██████████| 100/100 [2:48:43<00:00, 101.24s/it, train loss sum=4.53e+6, test acc=0.835]


In [ ]:
ResVAE = ResVAE.eval()
torch.save(ResVAE.state_dict(), '/content/drive/MyDrive/DL/Group/AdvPFY/model/cifar_resnet.pth')

In [ ]:
print(classification_report(gt_list, pred_list))
print(confusion_matrix(gt_list, pred_list))

              precision    recall  f1-score   support

           0       0.84      0.87      0.86      1000
           1       0.94      0.95      0.95      1000
           2       0.78      0.82      0.80      1000
           3       0.62      0.51      0.56      1000
           4       0.85      0.84      0.85      1000
           5       0.61      0.71      0.66      1000
           6       0.96      0.88      0.92      1000
           7       0.90      0.91      0.90      1000
           8       0.93      0.92      0.93      1000
           9       0.92      0.94      0.93      1000

    accuracy                           0.83     10000
   macro avg       0.84      0.83      0.83     10000
weighted avg       0.84      0.83      0.83     10000

[[870  11  20   8  18   5   1   9  34  24]
 [  3 953   0   1   1   0   0   0  10  32]
 [ 58   0 816  30  50  25   7  10   2   2]
 [  8   1  77 513  21 345  16   5   6   8]
 [ 32   0  51  15 842  17   6  36   1   0]
 [  7   1  33 206   6 712 

In [ ]:
# === 保存训练指标 ===
save_dir = '/content/drive/MyDrive/DL/Group/AdvPFY/results/'
os.makedirs(save_dir, exist_ok=True)

# === 保存训练指标 ===
report = classification_report(gt_list, pred_list, output_dict=True)
cm = confusion_matrix(gt_list, pred_list)

# 转为 DataFrame
df_report = pd.DataFrame(report).transpose()
df_cm = pd.DataFrame(cm)

# 保存为 CSV
df_report.to_csv(os.path.join(save_dir, 'train_classification_report.csv'), index=True)
df_cm.to_csv(os.path.join(save_dir, 'train_confusion_matrix.csv'), index=False, header=False)

print("✅ 训练指标已保存到：", save_dir)


✅ 训练指标已保存到： /content/drive/MyDrive/DL/Group/AdvPFY/results/


In [ ]:
# 加载权重
checkpoint_path = '/content/drive/MyDrive/DL/Group/AdvPFY/model/cifar_resnet.pth'
ResVAE.load_state_dict(torch.load(checkpoint_path, map_location=device))

# 进入评估模式
ResVAE = ResVAE.eval().to(device)


In [ ]:
# attack and purify
pred_list = []
pfy_pred_list = []
gt_list = []
ResVAE = ResVAE.eval()
for batch_idx, (data, target) in tqdm(enumerate(test_loader)):
    data, target = data.to(device), target.to(device)
    adv_vae = pgd_linf(data.to(device), target.to(device), ResVAE, atk_itr=16, eps=8/255, alpha=1/255, device=device)
    with torch.no_grad():
        y_test = ResVAE(adv_vae, deterministic=True, classification_only=True)

    purify_data_vae = vae_purify(adv_vae, ResVAE, atk_itr=32, eps=8/255, random_iteration=16, device=device)

    with torch.no_grad():
        pfy_y_test = ResVAE(purify_data_vae, deterministic=True, classification_only=True)

    pred_list += list(y_test.argmax(-1).cpu().detach().numpy())
    pfy_pred_list += list(pfy_y_test.argmax(-1).cpu().detach().numpy())
    gt_list += list(target.detach().cpu().numpy())

print('adversarial acc')
print(classification_report(gt_list, pred_list))
print(confusion_matrix(gt_list, pred_list))
print('purify acc')
print(classification_report(gt_list, pfy_pred_list))
print(confusion_matrix(gt_list, pfy_pred_list))

40it [2:36:43, 235.08s/it]

adversarial acc
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1000
           1       0.08      0.07      0.08      1000
           2       0.00      0.00      0.00      1000
           3       0.00      0.00      0.00      1000
           4       0.00      0.00      0.00      1000
           5       0.00      0.01      0.00      1000
           6       0.06      0.03      0.04      1000
           7       0.03      0.03      0.03      1000
           8       0.00      0.00      0.00      1000
           9       0.15      0.25      0.19      1000

    accuracy                           0.04     10000
   macro avg       0.03      0.04      0.03     10000
weighted avg       0.03      0.04      0.03     10000

[[  2  74 198  43  69 122  73  18 235 166]
 [ 37  75  48  48  38 127  37 118  51 421]
 [ 47  40   0   8  29 123  35 264  27 427]
 [ 86 103  20   0 305  48  37 356  10  35]
 [  2  32 187  89   0 415  45  15  78 137]
 [ 78 101 

In [ ]:
# === 保存对抗攻击与净化后的指标 ===
adv_report = classification_report(gt_list, pred_list, output_dict=True)
adv_cm = confusion_matrix(gt_list, pred_list)

pfy_report = classification_report(gt_list, pfy_pred_list, output_dict=True)
pfy_cm = confusion_matrix(gt_list, pfy_pred_list)

# 转为 DataFrame
df_adv_report = pd.DataFrame(adv_report).transpose()
df_adv_cm = pd.DataFrame(adv_cm)
df_pfy_report = pd.DataFrame(pfy_report).transpose()
df_pfy_cm = pd.DataFrame(pfy_cm)

# 保存为 CSV 文件
save_dir = '/content/drive/MyDrive/DL/Group/AdvPFY/results/'
df_adv_report.to_csv(save_dir + 'adv_classification_report.csv')
df_adv_cm.to_csv(save_dir + 'adv_confusion_matrix.csv', index=False, header=False)
df_pfy_report.to_csv(save_dir + 'purify_classification_report.csv')
df_pfy_cm.to_csv(save_dir + 'purify_confusion_matrix.csv', index=False, header=False)

print("✅ 攻击与净化指标已保存：")
print("  adv_classification_report.csv, adv_confusion_matrix.csv")
print("  purify_classification_report.csv, purify_confusion_matrix.csv")


✅ 攻击与净化指标已保存：
  adv_classification_report.csv, adv_confusion_matrix.csv
  purify_classification_report.csv, purify_confusion_matrix.csv
